### - Exercici 1
Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)





Seleccionem la web de quotes.toscrape i la pàgina de Wikipedia dels premis Oscar. Farem scraping primer amb Beautiful Soup a les dues i després farem el mateix amb Selenium.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

Creem una funció per fer scraping i iterar sobre totes les pàgines de "Next" per obtenir totes les cites, no només les de la primera pàgina.

In [2]:
def scrape_quotes(url):
    frases = []
    autors = []
    etiquetes = []

    while True:
        # Enviem una petició GET a la URL
        response = requests.get(url)

        # Creem un objecte BeautifulSoup per analitzar el contingut HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Trobem tots els elements div de les cites
        quotes = soup.find_all("div", class_="quote")

        # Iterem sobre cada cita
        for quote in quotes:
            # Extreiem el text de la cita
            frase = quote.find("span", class_="text").text

            # Extreiem l'autor de la cita
            autor = quote.find("small", class_="author").text

            # Extreiem les etiquetes de la cita
            etiqueta_tags = quote.find_all("a", class_="tag")
            etiquetes_quote = [tag.text for tag in etiqueta_tags]

            # Afegim la cita, l'autor i les etiquetes a les llistes corresponents
            frases.append(frase)
            autors.append(autor)
            etiquetes.append(etiquetes_quote)

        # Comprovem si hi ha una pàgina següent
        next_page = soup.find("li", class_="next")
        if next_page is None:
            # Si no hi ha una pàgina següent, surtim del bucle
            break
        else:
            # Si hi ha una pàgina següent, construim la URL de la pàgina següent
            next_url = urljoin(url, next_page.find("a")["href"])
            url = next_url

    # Creem un diccionari amb les dades obtingudes
    data = {"Frases": frases, "Autors": autors, "Etiquetes": etiquetes}

    # Creem un DataFrame a partir del diccionari de dades
    quotes_df = pd.DataFrame(data)

    # Retornem el DataFrame
    return quotes_df

# URL inicial
url = "http://quotes.toscrape.com"

# Escrapejem totes les pàgines
all_quotes_df = scrape_quotes(url)
all_quotes_df

,Frases,Autors,Etiquetes
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


Per últim, ho guardarem en un arxiu de CSV

In [3]:
all_quotes_df.to_csv("quotes.csv", index=False)

Podem observar que s'han extret les cites amb el nom del corresponent autor i les etiquetes. Tot això ho hem posat en una taula i ja estaria seria un dataset llest per utilitzar. Ara fem el mateix amb la Wikipedia dels premis Oscars. En aquest cas voldrem extreure una part de la taula i configurar-la amb el número de l'edició dels Oscars, la data en què es va celebrar i la pel·lícula guanyadora del premi a millor pel·lícula.

Fem la sol·licitud GET a l'enllaç de la Wikipedia

In [4]:
url = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"
response = requests.get(url)

Creem un objecte de Beautiful Soup amb el contingut de l'HTML

In [5]:
soup = BeautifulSoup(response.content, "html.parser")

Busquem la taula que conté els guanyadors

In [6]:
taula = soup.find("table", class_="wikitable")

Creem listes buides per guardar les dades

In [7]:
edicions = []
dates = []
pelicules = []

Extreiem les dades de les files de la taula

In [8]:
files = taula.find_all("tr")[1:]  

In [9]:
for fila in files:
    celes = fila.find_all("td")
    edicio = celes[0].text.strip()
    data = celes[1].text.strip()
    pelicula = celes[2].text.strip()

    edicions.append(edicio)
    dates.append(data)
    pelicules.append(pelicula)

Creem un dataframe amb les dades i l'imprimim

In [10]:
data = {"Edició": edicions, "Data": dates, "Millor pel·lícula": pelicules}
oscars_df = pd.DataFrame(data)
oscars_df

,Edició,Data,Millor pel·lícula
0,1°,16 de mayode 1929,Alas (Wings) (1927)
1,2°,3 de abrilde 1930,La melodía de Broadway (The Broadway melody) (...
2,3°,5 de noviembrede 1930,Sin novedad en el frente (All quiet on the wes...
3,4°,10 de noviembrede 1931,Cimarron (1931)
4,5°,18 de noviembrede 1932,Grand Hotel
...,...,...,...
90,91°,24 de febrerode 2019,Green Book
91,92°,9 de febrerode 2020,Parasite
92,93°,25 de abrilde 2021,Nomadland
93,94°,27 de marzode 2022,CODA


Ara ho farem amb Selenium:

Creem un DataFrame amb els elements i l'imprimim

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# Configuració de les opcions del ChromeDriver
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# URL inicial
url = 'http://quotes.toscrape.com'
frases = []
autors = []
etiquetes = []

while True:
    # Obtenim la pàgina amb Selenium
    driver.get(url)

    # Trobem tots els elements de les cites
    quotes = driver.find_elements(By.CSS_SELECTOR, '.quote')

    # Iterem sobre cada cita
    for quote in quotes:
        # Extreiem el text de la cita
        frase = quote.find_element(By.CSS_SELECTOR, '.text').text

        # Extreiem l'autor de la cita
        autor = quote.find_element(By.CSS_SELECTOR, '.author').text

        # Extreiem les etiquetes de la cita
        etiquetes_quote = [tag.text for tag in quote.find_elements(By.CSS_SELECTOR, '.tag')]

        # Afegim la cita, l'autor i les etiquetes a les llistes corresponents
        frases.append(frase)
        autors.append(autor)
        etiquetes.append(etiquetes_quote)

    try:
        # Trobem el botó de la pàgina següent
        next_button = driver.find_element(By.CSS_SELECTOR, 'li.next a')
        url = next_button.get_attribute('href')
    except NoSuchElementException:
        # Si no hi ha un botó de la pàgina següent, surtim del bucle
        break

# Tanquem la finestra del navegador
driver.quit()

# Creem un diccionari amb les dades obtingudes
data = {"Frases": frases, "Autors": autors, "Etiquetes": etiquetes}

# Creem un DataFrame a partir del diccionari de dades
all_quotes_sel = pd.DataFrame(data)

# Mostrem el DataFrame
all_quotes_sel

,Frases,Autors,Etiquetes
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


Ara fem el mateix amb la pàgina consultada de la Wikipedia

In [13]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [14]:
url = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"
driver.get(url)

In [15]:
tabla = driver.find_element(By.CSS_SELECTOR, ".wikitable")

In [16]:
edicions = []
dates = []
pelicules = []

In [17]:
files = tabla.find_elements(By.TAG_NAME, "tr")[1:]

In [18]:
for fila in files:
    celes = fila.find_elements(By.TAG_NAME, "td")
    edicio = celes[0].text.strip()
    data = celes[1].text.strip()
    pelicula = celes[2].text.strip()

    edicions.append(edicio)
    dates.append(data)
    pelicules.append(pelicula)

In [19]:
driver.quit()

In [20]:
data = {"Edició": edicions, "Data": dates, "Millor pel·lícula": pelicules}
oscars_df2 = pd.DataFrame(data)
oscars_df2

,Edició,Data,Millor pel·lícula
0,1°,16 de mayo\nde 1929,Alas (Wings) (1927)
1,2°,3 de abril\nde 1930,La melodía de Broadway (The Broadway melody) (...
2,3°,5 de noviembre\nde 1930,Sin novedad en el frente (All quiet on the wes...
3,4°,10 de noviembre\nde 1931,Cimarron (1931)
4,5°,18 de noviembre\nde 1932,Grand Hotel
...,...,...,...
90,91°,24 de febrero\nde 2019,Green Book
91,92°,9 de febrero\nde 2020,Parasite
92,93°,25 de abril\nde 2021,Nomadland
93,94°,27 de marzo\nde 2022,CODA


Veiem que amb Selenium s'utilitzen més linies de codi que amb BeautifulSoup i que Selenium és una mica menys intuitiu, però és útil per fer scraping de webs dinàmiques, pel contrari que BeautifulSoup que només ho fa al codi HTML de les webs estàtiques. 

Selenium és molt útil per interectuar amb el navegador i el contingut generat amb JavaScript, d'aquesta manera permet fer clic als elements, omplir formularis i desplaçar-se per la pàgina, entre d'altres. Això és molt útil quan el contingut no està present al codi HTML inicial sinó que es carrega i actualitza de manera dinàmica. 

### - Exercici 2
Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

#### Títol: 
Quotes to Scrape
#### Sobre el context del dataset: 
Aquesta web conté cites de personalitats reconegudes. 
#### Contingut:
A cada caixa de text s'indica la cita completa, el nom de l'autor a sota i les etiquetes relacionades amb cada cita. En total hi ha 10 frases a la primera pàgina, però les pàgines es poden anar passant i veiem que en total n'hi ha 99. 

___________________________________________

#### Títol:  
Premios Óscar
#### Sobre el context del dataset: 
Aquesta pàgina de la Wikipedia parla de la cerimònia dels premis Òscar de manera extensa i posant en context la seva celebració així com tota mena de dades i curiositats. 
#### Contingut:
L'índex de l'article seria aquest.

1. Premios
2. Historia
3. Categorías
4. Ceremonias
5. Estadísticas y récords
6. Curiosidades
7. Críticas y controversias
8. Véase también
9. Referencias
10. Enlaces externos


### - Exercici 3
Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després. 

Faré scraping a la web de Formula TV per obtenir el text relacionat i l'ordre d'actuacions d'Eurovisió que és properament.

In [21]:
driver = webdriver.Chrome()

In [22]:
driver.get("https://www.formulatv.com/noticias/blanca-paloma-actuara-octava-final-eurovision-122522/")

Busquem els elements < p >

In [23]:
p_elements = driver.find_elements(By.TAG_NAME, 'p')

Imprimim el text dels elements eliminant els espais en blanc

In [24]:
for element in p_elements:
    text = element.text.strip()  
    if text:  
        print(text)
        print()

LISTA COMPLETA

Tras la celebración de las semifinales, ya conocemos qué países actuarán en la gala que se celebrará el próximo sábado.

Programa relacionado

Festival de Eurovisión 2023

2023

Reino Unido

Concursos Talent

La Gran Final del Festival de Festival de Eurovisión 2023 está a la vuelta de la esquina. El próximo sábado 13 de mayo conoceremos quién se hará con el deseado micrófono de cristal y, tras la celebración de las dos semifinales, ya contamos con la lista completa de los veintiséis países que se lo disputarán y en qué orden actuarán.

El país encargado de arraigar la Gran Final será los representantes de Austria, Teya & Salena, con la divertida "Who The Hell Is Edgar?", mientras que el país anfitrión (que no ganador), Reino Unido, con Mae Muller y su "I Wrote A Song" la cerrarán. Blanca Paloma, por su parte, hará sonar su "Eaea" sobre el escenario del M&S Bank Arena de Liverpool en octava posición.

Blanca Paloma durante uno ensayo del Festival de Eurovisión 2023

1.-

In [25]:
driver.quit()